In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from pprint import pprint

In [9]:
# Source: https://towardsdatascience.com/how-to-develop-a-credit-risk-model-and-scorecard-91335fc01f03
# https://github.com/finlytics-hub/credit_risk_model
def run_query(query):

    # endpoint for making the request
    # https://api.thegraph.com/subgraphs/name/aave/protocol
    # https://api.thegraph.com/subgraphs/name/aave/protocol-multy-raw
    request = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

In [45]:
users_query = """
{
  users(where: {
    id_not_in: ["0x0000000000000000000000000000000000000001",
                "0x000000000000000000000000000000000000dead"]
  }){
    id
    unclaimedRewards
    lifetimeRewards
    depositHistory(orderBy: timestamp, orderDirection: desc,) {
      id
      amount
      timestamp
      reserve {
        symbol
        name
      }
    }
    borrowHistory(orderBy: timestamp, orderDirection: desc) {
      id
      amount
      timestamp
      reserve {
        symbol
        name
      }
      borrowRate
      borrowRateMode
      stableTokenDebt
      variableTokenDebt
    }
    repayHistory{
      id
      amount
      timestamp
      reserve{
        symbol
        name
      }
    }
  }
}
"""


In [46]:
users_result = run_query(users_query)
#pprint(user_result)
users_result['data']['users']

[{'borrowHistory': [],
  'depositHistory': [{'amount': '127090210000000016',
    'id': '0x27a9c85712e3ee4ec50e6a36b37bda680e0625f05ccccd83f5ce1968683197ef:1',
    'reserve': {'name': 'Wrapped Ether', 'symbol': 'WETH'},
    'timestamp': 1626329888},
   {'amount': '115695250000000000',
    'id': '0xe717d4ca0781abd0794181afd02e73b639624128c580d4bbc21ab8765eb80de0:1',
    'reserve': {'name': 'Wrapped Ether', 'symbol': 'WETH'},
    'timestamp': 1626201044},
   {'amount': '115019330000000000',
    'id': '0x463748b8a596ea55f10ad80c364cbfc98a7747235a42b7cca7f8e9e3bf98c7f1:1',
    'reserve': {'name': 'Wrapped Ether', 'symbol': 'WETH'},
    'timestamp': 1626195622},
   {'amount': '132166879999999984',
    'id': '0x5d696844123882718dfc840dfdfa9fb8eb0c45c0945455454a6b520c60cc81f1:1',
    'reserve': {'name': 'Wrapped Ether', 'symbol': 'WETH'},
    'timestamp': 1626074200},
   {'amount': '114580540000000000',
    'id': '0x0c995a5df12b87c3da2f4eabc954183f5522bd27ec596abd6c415c44161cb71b:1',
    'rese

In [47]:
def extract_deposit_history(user_id, deposit=None):
    """
    Extract deposit history from a given user
    """
    
    deposit_dict = dict(
        user_id = user_id,
        deposit_reserve_name = '',
        deposit_reserve_symbol = '',
        deposit_amount = 0,
        deposit_timestamp_unix = None,
        deposit_timestamp = None
    )
    
    if deposit:
        deposit_dict['deposit_reserve_name'] = deposit['reserve']['name']
        deposit_dict['deposit_reserve_symbol'] = deposit['reserve']['symbol']
        deposit_dict['deposit_amount'] = int(deposit['amount'])
        deposit_dict['deposit_timestamp_unix'] = deposit['timestamp']
        deposit_dict['deposit_timestamp'] = pd.to_datetime(deposit['timestamp'], 
                                                           unit='s', errors='ignore')
        
    return deposit_dict

def extract_borrow_history(user_id, borrow=None):
    """
    Extract borrow history from a given user
    """
    borrow_dict = dict(
        user_id = user_id,
        borrow_reserve_name = '',
        borrow_reserve_symbol = '',
        borrow_amount = 0,
        borrow_timestamp_unix = None,
        borrow_timestamp = None,
        borrow_rate = 0,
        borrow_rate_mode = '',
        borrow_stableTokenDebt = 0,
        borrow_variableTokenDebt = 0
        #borrow_accrued_interest = 0
    )
    
    if borrow:
        borrow_dict['borrow_reserve_name'] = borrow['reserve']['name']
        borrow_dict['borrow_reserve_symbol'] = borrow['reserve']['symbol']
        borrow_dict['borrow_amount'] = int(borrow['amount'])
        borrow_dict['borrow_timestamp_unix'] = borrow['timestamp']
        borrow_dict['borrow_timestamp'] = pd.to_datetime(borrow['timestamp'],
                                                         unit='s', errors='ignore')
        borrow_dict['borrow_rate'] = int(borrow['borrowRate'])
        borrow_dict['borrow_rate_mode'] = borrow['borrowRateMode']
        borrow_dict['borrow_stableTokenDebt'] = borrow['stableTokenDebt']
        borrow_dict['borrow_variableTokenDebt'] = borrow['variableTokenDebt']
        #borrow_dict['borrow_accrued_interest'] = int(borrow['accruedBorrowInterest'])
    
    return borrow_dict

def extract_repay_history(user_id, repay=None):
    
    repay_dict = dict(
        user_id = user_id,
        repay_reserve_name = '',
        repay_reserve_symbol = '',
        repay_amount = 0,
        repay_timestamp_unix = None,
        repay_timestamp = None
    )
    
    if repay:
        repay_dict['repay_reserve_name'] = repay['reserve']['name']
        repay_dict['repay_reserve_symbol'] = repay['reserve']['symbol']
        repay_dict['repay_amount'] = int(repay['amount'])
        repay_dict['repay_timestamp_unix'] = repay['timestamp']
        repay_dict['repay_timestamp'] = pd.to_datetime(repay['timestamp'],
                                                       unit='s', errors='ignore')

    return repay_dict


In [49]:
users, deposits, borrows, repays = [], [], [], []
for user in users_result['data']['users']:
    #print(user, '\n')
    
    user_id = user['id']
    
    users.append(dict(
        user_id = user_id,
        user_unclaimedRewards = user['unclaimedRewards'],
        user_lifetimeRewards = user['lifetimeRewards']
    ))
    
    # extract depositHistory
    if user['depositHistory']:
        for deposit in user['depositHistory']:
            deposit_dict = extract_deposit_history(user_id, deposit)
            deposits.append(deposit_dict)
    else:
        deposit_dict = extract_deposit_history(user_id)
        deposits.append(deposit_dict)
    
    # extract borrowHistory
    if user['borrowHistory']:
        for borrow in user['borrowHistory']:
            borrow_dict = extract_borrow_history(user_id, borrow)
            borrows.append(borrow_dict)
    else:
        borrow_dict = extract_borrow_history(user_id)
        borrows.append(borrow_dict)
   
    # extract repayHistory
    if user['repayHistory']:
        for repay in  user['repayHistory']:
            repay_dict = extract_repay_history(user_id, repay)
            repays.append(repay_dict)
    else:
        repay_dict = extract_repay_history(user_id)
        repays.append(repay_dict)
        
            

In [27]:
users_df = pd.DataFrame(users)
users_df.head()

,user_id,user_unclaimedRewards,user_lifetimeRewards
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,46,46
1,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,7217726631361778,7217726631361778
2,0x00000000005dbcb0d0513fcda746382fe8a53468,0,0
3,0x00000000009a41862f3b2b0c688b7c0d1940511e,0,0
4,0x0000000000d41c96294ccdac8612bdfe29c641af,0,0


In [18]:
deposits_df = pd.DataFrame(deposits)
deposits_df.head()

,user_id,deposit_reserve_name,deposit_reserve_symbol,deposit_amount,deposit_timestamp_unix,deposit_timestamp
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,127090210000000016,1.626330e+09,2021-07-15 06:18:08
1,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,115695250000000000,1.626201e+09,2021-07-13 18:30:44
2,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,115019330000000000,1.626196e+09,2021-07-13 17:00:22
3,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,132166879999999984,1.626074e+09,2021-07-12 07:16:40
4,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,114580540000000000,1.625835e+09,2021-07-09 12:50:26


In [43]:
borrows_df = pd.DataFrame(borrows)
borrows_df.head()

,user_id,borrow_reserve_name,borrow_reserve_symbol,borrow_amount,borrow_timestamp_unix,borrow_timestamp,borrow_rate,borrow_rate_mode,borrow_stableTokenDebt,borrow_variableTokenDebt
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,,,0,NaN,NaT,0,,0,0
1,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Synthetix Network Token,SNX,5000000000000000000000,1.622545e+09,2021-06-01 11:00:43,127312792424753718439733675,Variable,0,4760354075421655119279
2,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Republic Token,REN,100000000000000000000000,1.622303e+09,2021-05-29 15:35:30,8024003551450866705556542,Variable,0,98740619248157900299254
3,0x00000000005dbcb0d0513fcda746382fe8a53468,,,0,NaN,NaT,0,,0,0
4,0x00000000009a41862f3b2b0c688b7c0d1940511e,,,0,NaN,NaT,0,,0,0


In [44]:
no_deposits = set(deposits_df[deposits_df.deposit_amount == 0].user_id)
no_borrows = set(borrows_df[borrows_df.borrow_amount == 0].user_id)

print(len(no_deposits))
print(len(no_borrows))

len(no_deposits.intersection(no_borrows))

23
69


16

In [52]:
repays_df = pd.DataFrame(repays)
repays_df.head(50)

,user_id,repay_reserve_name,repay_reserve_symbol,repay_amount,repay_timestamp_unix,repay_timestamp
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,,,0,NaN,NaT
1,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Synthetix Network Token,SNX,5000478484297793675639,1.622568e+09,2021-06-01 17:21:43
2,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Republic Token,REN,100000840317343090649398,1.622335e+09,2021-05-30 00:42:31
3,0x00000000005dbcb0d0513fcda746382fe8a53468,,,0,NaN,NaT
4,0x00000000009a41862f3b2b0c688b7c0d1940511e,,,0,NaN,NaT
5,0x0000000000d41c96294ccdac8612bdfe29c641af,,,0,NaN,NaT
6,0x000000000a38444e0a6e37d3b630d7e855a7cb13,Ampleforth,AMPL,1237311201614,1.627464e+09,2021-07-28 09:25:22
7,0x000000003ce0cf2c037493b1dc087204bd7f713e,,,0,NaN,NaT
8,0x0000000094acb89a43eac2fbb3a07973efc2435c,,,0,NaN,NaT
9,0x0000000484f2217f1a64eb6d24b5cee446faeae5,Wrapped Ether,WETH,300002875869535989282,1.617127e+09,2021-03-30 17:55:39
